In [8]:
import os
import json
import pandas as pd

# 파일 경로 설정
stats_dir = "./world_copy/stats"
whitelist_file = "./whitelist_copy.json"

# whitelist.json 로드
with open(whitelist_file, "r", encoding="utf-8") as file:
    whitelist_data = json.load(file)

# UUID와 닉네임 매핑
uuid_to_name = {entry["uuid"]: entry["name"] for entry in whitelist_data}

# 플레이어 통계 저장용 리스트
stats_list = []

# stats 폴더 내 JSON 파일 읽기
for filename in os.listdir(stats_dir):
    if filename.endswith(".json"):  # JSON 파일만 처리
        file_path = os.path.join(stats_dir, filename)
        player_uuid = filename.replace(".json", "")  # 파일명에서 UUID 추출

        with open(file_path, "r", encoding="utf-8") as file:
            try:
                data = json.load(file)  # JSON 파싱
                
                # 주요 통계 정보 추출 (예: 점프 횟수, 채굴 블록 수, 플레이 시간)
                stats = data.get("stats", {}).get("minecraft:custom", {})
                play_time = stats.get("minecraft:play_one_minute", 0) // 20  # 틱 → 초 변환
                jumps = stats.get("minecraft:jump", 0)
                blocks_mined = stats.get("minecraft:mined", {})

                # 데이터 정리
                stats_list.append({
                    "uuid": player_uuid,
                    "name": uuid_to_name.get(player_uuid, "Unknown"),
                    "play_time_sec": play_time,
                    "jumps": jumps,
                    "blocks_mined": sum(blocks_mined.values())  # 모든 블록 채굴 합산
                })
            except json.JSONDecodeError:
                print(f"Error decoding JSON in file: {filename}")

# 데이터프레임 생성
df_stats = pd.DataFrame(stats_list)
df_stats

,uuid,name,play_time_sec,jumps,blocks_mined
0,0cae8065-4520-42aa-995f-6aafa82fb817,ventulus95,0,991,0
1,11ac9b52-c891-4032-9624-f5e253dd9f39,Unknown,0,1,0
2,383d2ff8-be20-4c83-ad3e-49f8d09f3bc2,G_NI,0,7079,0
3,4058108e-9065-45a9-b914-9d9a9d93c40b,NamiHam,0,10830,0
4,569ac8f8-e0f0-4712-b8ab-376be24ad11d,YulBae,0,25558,0
5,583e2056-1ca7-4ea5-8c6a-94714ea08a41,gogogonini,0,1097,0
6,5d89f372-53b4-48eb-ab26-33411fd1fed9,bossminji,0,8154,0
7,6161833d-988d-470b-bc3b-7cd91ff4fe68,AugustusJihanVI,0,15209,0
8,80b45824-0a21-4947-96b9-4875a67a4f62,nasirK17,0,11244,0
9,8f6a4f95-0c37-4eb7-b93c-b487dc7e5673,teddygood,0,1289,0


In [ ]:
import os
import json
import pandas as pd

# whitelist 파일 로드
whitelist_file = "./whitelist_copy.json"
with open(whitelist_file, "r", encoding="utf-8") as file:
    whitelist_data = json.load(file)

# UUID와 닉네임 매핑
uuid_to_name = {entry["uuid"]: entry["name"] for entry in whitelist_data}

# stats 폴더 설정
stats_dir = "./world_copy/stats"

# 파일 처리
for filename in os.listdir(stats_dir):
    if filename.endswith(".json"):  # JSON 파일만 처리
        file_path = os.path.join(stats_dir, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)  # JSON 파싱
            
            # UUID 추출 후 닉네임 매핑
            player_uuid = filename.replace(".json", "")
            player_name = uuid_to_name.get(player_uuid, "Unknown")

            # 데이터 변환을 위한 리스트
            categories = {
                "minecraft:killed": [],
                "minecraft:picked_up": [],
                "minecraft:custom": [],
                "minecraft:crafted": [],
                "minecraft:mined": [],
                "minecraft:used": [],
                "minecraft:dropped": [],
                "minecraft:broken": []
            }

            # 각 카테고리별 데이터 변환
            for category, values in data["stats"].items():
                if category in categories:
                    for item, count in values.items():
                        clean_item = item.replace("minecraft:", "")  # 접두사 제거
                        categories[category].append({
                            "Player": player_name,  # 닉네임 추가
                            "Category": category.replace("minecraft:", ""),  # 접두사 제거
                            "Item": clean_item,
                            "Count": count
                        })

            # 데이터프레임 생성
            df_dict = {key.replace("minecraft:", ""): pd.DataFrame(value) for key, value in categories.items()}
            
            # 예시로 'killed' 데이터 출력
            display(df_dict["killed"])
            break  # 첫 번째 파일만 처리 후 종료 (테스트 목적)

,Player,Category,Item,Count
0,ventulus95,killed,pig,1
1,ventulus95,killed,spider,2
2,ventulus95,killed,bat,1
3,ventulus95,killed,skeleton,1
4,ventulus95,killed,zombie,1
5,ventulus95,killed,chicken,7


In [62]:
# ID와 실제 이름 매핑 딕셔너리
uuid_name_map = {
    "integerji": "지정수",
    "belleoque7": "임정",
    "jung_ggplot": "임정",
    "gogogonini": "박건희",
    "bossminji": "우민지a",
    "hojongs": "전종호",
    "YulBae": "배성율",
    "G_NI": "송태현",
    "MakeURegret": "MakeURegret",
    "imnotheomin": "허민(글또 9기)",
    "nasirK17": "김종진",
    "AugustusJihanVI": "박지한",
    "ventulus95": "이창섭",
    "teddygood": "이찬호",
    "KangDAEHYUP": "강승현",
    "suminping": "김수민b",
    "nakjunnakjun": "황낙준",
    "NamiHam": "남희정"
}

whitelist_file = "./whitelist_copy.json"
if os.path.exists(whitelist_file):
    with open(whitelist_file, "r", encoding="utf-8") as file:
        whitelist_data = json.load(file)
    whitelist_uuid_map = {entry["uuid"]: entry["name"] for entry in whitelist_data}
else:
    whitelist_uuid_map = {}

# stats 폴더 설정
stats_dir = "./world_copy/stats"

# 전체 데이터를 저장할 딕셔너리
all_data = {
    "killed": [],
    "picked_up": [],
    "custom": [],
    "crafted": [],
    "mined": [],
    "used": [],
    "dropped": [],
    "broken": []
}

# stats 폴더 내 모든 JSON 파일 처리
for filename in os.listdir(stats_dir):
    if filename.endswith(".json"):  # JSON 파일만 처리
        file_path = os.path.join(stats_dir, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)  # JSON 파싱

            # UUID 추출 후 닉네임 매핑
            player_uuid = whitelist_uuid_map.get(filename.replace(".json", ""), filename.replace(".json", ""))
            player_name = uuid_name_map.get(player_uuid, "Unknown")

            # 각 카테고리별 데이터 변환
            for category, values in data["stats"].items():
                category_cleaned = category.replace("minecraft:", "")  # 접두사 제거
                if category_cleaned in all_data:
                    for item, count in values.items():
                        clean_item = item.replace("minecraft:", "")  # 접두사 제거
                        all_data[category_cleaned].append({
                            "Player": player_name,  # 닉네임 추가
                            "Category": category_cleaned,
                            "Item": clean_item,
                            "Count": count
                        })

# 데이터프레임 생성
df_dict = {key: pd.DataFrame(value) for key, value in all_data.items()}

- killed	플레이어가 처치한 몹 (몬스터, 동물 등)
- picked_up	플레이어가 줍거나 획득한 아이템
- custom	특정 행동(점프, 이동 거리, 플레이 시간 등)
- crafted	플레이어가 제작한 아이템
- mined	플레이어가 채굴한 블록
- used	플레이어가 사용한 아이템 (ex. 검, 곡괭이, 음식)
- dropped	플레이어가 바닥에 버린 아이템
- broken	플레이어가 부순 도구 또는 블록

In [63]:
# 통합 CSV 파일로 저장
combined_csv_filename = "minecraft_stats_combined.csv"

# 모든 데이터프레임을 하나로 합치기 위해 리스트 생성
df_list = []
for key, df in df_dict.items():
    df["Category"] = key  # 각 데이터프레임에 'Category' 컬럼 추가
    df_list.append(df)

# 모든 데이터프레임을 하나로 합치기
df_combined = pd.concat(df_list, ignore_index=True)

# CSV 파일로 저장
df_combined.to_csv(combined_csv_filename +'250202', index=False, encoding="utf-8-sig")

In [57]:
df_dict['picked_up']

,Player,Category,Item,Count
0,이창섭,picked_up,birch_log,19
1,이창섭,picked_up,crafting_table,6
2,이창섭,picked_up,wheat,3
3,이창섭,picked_up,feather,5
4,이창섭,picked_up,chicken,7
...,...,...,...,...
2020,임정,picked_up,black_wool,2
2021,임정,picked_up,warped_nylium,3
2022,임정,picked_up,coal,189
2023,임정,picked_up,sunflower,13


In [55]:
# 데이터프레임이 이미 존재한다고 가정하고 처리
import pandas as pd

# 몬스터와 동물 리스트 정의 (예제)
monsters = {"zombie", "skeleton", "creeper", "spider", "blaze", "enderman", "witch", "vindicator", "phantom"}
animals = {"cow", "chicken", "pig", "sheep", "horse", "turtle", "rabbit", "fox", "wolf", "cat", "bee"}

# 몬스터와 동물을 죽인 횟수를 집계
df_monster_kills = df_dict["killed"][df_dict["killed"]["Item"].isin(monsters)].groupby("Player")["Count"].sum()
df_animal_kills = df_dict["killed"][df_dict["killed"]["Item"].isin(animals)].groupby("Player")["Count"].sum()

# 몬스터를 가장 많이 죽인 사람
top_monster_killer = df_monster_kills.idxmax() if not df_monster_kills.empty else "No data"
top_monster_kills = df_monster_kills.max() if not df_monster_kills.empty else 0

# 동물을 가장 많이 죽인 사람
top_animal_killer = df_animal_kills.idxmax() if not df_animal_kills.empty else "No data"
top_animal_kills = df_animal_kills.max() if not df_animal_kills.empty else 0

# 결과 출력
top_monster_killer, top_monster_kills, top_animal_killer, top_animal_kills

('지정수', np.int64(3528), '지정수', np.int64(474))

In [56]:
df_dict['picked_up']

,Player,Category,Item,Count
0,이창섭,picked_up,birch_log,19
1,이창섭,picked_up,crafting_table,6
2,이창섭,picked_up,wheat,3
3,이창섭,picked_up,feather,5
4,이창섭,picked_up,chicken,7
...,...,...,...,...
2020,임정,picked_up,black_wool,2
2021,임정,picked_up,warped_nylium,3
2022,임정,picked_up,coal,189
2023,임정,picked_up,sunflower,13


In [44]:
whitelist_uuid_map[player_uuid]

'jung_ggplot'